<a href="https://colab.research.google.com/github/nyuNIX/Sentinel-Download/blob/main/Sentinel2_Talhoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para rodar essa parte tem-se que fazer a conexão de modo interativa com o drive.

O mesmo se aplica ao projeto do GEE da sua conta (já abilitado a conexão com o sentinel-2)

In [ ]:
# =============================================================================
#  - Monta o Google Drive
#  - Instala dependências geoespaciais (necessárias apenas para SHP)
#  - Autentica e inicializa o Google Earth Engine
# =============================================================================

from google.colab import drive
drive.mount('/content/drive')

# Dependências necessárias apenas se forem usados shapefiles (.shp ou .zip).
# GeoJSON puro não exige geopandas, mas manter aqui evita erros futuros.
!pip install --quiet geopandas fiona shapely pyproj rtree

import os
import json
import zipfile
import tempfile
import ee

# Autenticação interativa do Earth Engine (modo usuário)
ee.Authenticate()   # abrirá um link e solicitará o token
ee.Initialize(project='sentinel-479818')

print("Drive montado e Earth Engine inicializado.")


cloudProbThreshold e maxAllowedCloudPercentage são valores que podem ser alterados conforme a necessidade. Dependem do modelo e do que buscamos inferir  

In [ ]:
# Intervalo temporal de busca das imagens Sentinel-2
start_Date = '2024-01-01'
end_Date   = '2024-01-10'

# Threshold (%) de probabilidade de nuvem (S2 Cloud Probability)
# Pixels acima desse valor são considerados nuvem
cloudProbThreshold = 50

# Porcentagem (%) de nuvem aceitavel dentro do talhão
# Se dentro do talhão tiver um valor maior ou igual a X% ele ainda é coletado
maxAllowedCloudPercentage = 0.1  # 10%

# Bandas a serem exportadas (RGB)
bands = ['B2', 'B3', 'B4']

# Limite máximo de pixels permitido por export (Earth Engine safeguard)
maxPixels = 1e13

# Diretório no Google Drive contendo os polígonos de entrada
talhoes_dir = '/content/drive/MyDrive/GEE_EXPORT/Talhoes'

# Pasta base de saída no Google Drive
# Subpastas serão criadas automaticamente para cada polígono
drive_export_folder = 'Dataset-output'
base_output_path = f"/content/drive/MyDrive/GEE_EXPORT/{drive_export_folder}"

print("Configurações carregadas.")
print("Procurando polígonos em:", talhoes_dir)


In [ ]:
import geopandas as gpd

def load_fc_from_shp(shp_path):
    """
    Lê um shapefile local e o converte para ee.FeatureCollection.
    Caso o CRS não seja EPSG:4326, a reprojeção é aplicada.
    """
    gdf = gpd.read_file(shp_path)
    if gdf.crs and gdf.crs.to_string() != 'EPSG:4326':
        gdf = gdf.to_crs(epsg=4326)
    return ee.FeatureCollection(json.loads(gdf.to_json()))


def load_all_polygons_from_drive_folder(folder_path):
    """
    Varre uma pasta do Google Drive e carrega todos os polígonos encontrados,
    suportando os seguintes formatos:
      1) GeoJSON / JSON
      2) Shapefiles soltos (.shp)
      3) Shapefiles compactados (.zip)

    Retorna
    -------
    list of tuples
        Lista de pares (nome_base, ee.FeatureCollection)
    """
    items = []

    if not os.path.exists(folder_path):
        raise FileNotFoundError("Pasta não encontrada: " + folder_path)

    files = os.listdir(folder_path)

    # -------------------------------------------------------------------------
    # 1) GeoJSON / JSON
    # -------------------------------------------------------------------------
    for f in files:
        if f.lower().endswith(('.geojson', '.json')):
            name = os.path.splitext(f)[0]
            path = os.path.join(folder_path, f)
            with open(path, 'r', encoding='utf-8') as fp:
                gj = json.load(fp)
            items.append((name, ee.FeatureCollection(gj)))
            print("Carregado GeoJSON:", f)

    # -------------------------------------------------------------------------
    # 2) Shapefiles soltos (.shp)
    # -------------------------------------------------------------------------
    shp_files = [f for f in files if f.lower().endswith('.shp')]
    for shp in shp_files:
        name = os.path.splitext(shp)[0]
        shp_path = os.path.join(folder_path, shp)
        try:
            fc = load_fc_from_shp(shp_path)
            items.append((name, fc))
            print("Carregado SHP:", shp)
        except Exception as e:
            print("Erro ao carregar SHP", shp, "->", e)

    # -------------------------------------------------------------------------
    # 3) Shapefiles compactados (.zip)
    # -------------------------------------------------------------------------
    for f in files:
        if f.lower().endswith('.zip'):
            name = os.path.splitext(f)[0]
            zip_path = os.path.join(folder_path, f)
            try:
                with zipfile.ZipFile(zip_path, 'r') as z:
                    tmpdir = tempfile.mkdtemp()
                    z.extractall(tmpdir)

                shp_inside = [
                    os.path.join(tmpdir, x)
                    for x in os.listdir(tmpdir)
                    if x.lower().endswith('.shp')
                ]

                if shp_inside:
                    fc = load_fc_from_shp(shp_inside[0])
                    items.append((name, fc))
                    print("Carregado SHP ZIP:", f)

            except Exception as e:
                print("Erro ao carregar ZIP", f, "->", e)

    return items


# Execução do carregamento
polygons_list = load_all_polygons_from_drive_folder(talhoes_dir)
print("Total de FeatureCollections carregadas:", len(polygons_list))


Neste processamento, todas as imagens Sentinel-2 que intersectam a geometria de interesse são mantidas, mesmo quando pertencem à mesma data de aquisição. Isso ocorre porque o Sentinel-2 é organizado em tiles MGRS independentes, e um mesmo polígono pode estar localizado na sobreposição de múltiplos tiles. Nesses casos, produtos adquiridos no mesmo instante podem apresentar condições distintas de cobertura de nuvens sobre a área de interesse, de modo que descartar imagens apenas com base na data poderia resultar na perda de informação válida. A manutenção de todos os produtos introduz uma **redundância positiva**, aumentando a robustez do conjunto de dados e reduzindo lacunas causadas por nuvens.

In [ ]:
def add_cloud_probability_join(s2_col, cloud_col):
    """
    Associa a cada imagem Sentinel-2 SR a respectiva imagem de
    probabilidade de nuvem (S2Cloudless) utilizando um join exato
    baseado na propriedade 'system:index'.

    Esta é a abordagem oficialmente recomendada pelo Google Earth Engine.

    https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    """

    joined = ee.Join.saveFirst('cloud_prob_img').apply(
        primary=s2_col,
        secondary=cloud_col,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    )

    def _append_cloud_band(img):
        cloud_img = ee.Image(img.get('cloud_prob_img'))

        # Proteção defensiva (caso raro de imagem sem par)
        cloud_prob = ee.Algorithms.If(
            cloud_img,
            cloud_img.select('probability').rename('cloud_prob'),
            ee.Image.constant(0).rename('cloud_prob')
        )

        return ee.Image(img).addBands(ee.Image(cloud_prob))

    return ee.ImageCollection(joined).map(_append_cloud_band)




def process_polygon_fc(name, fc, startDate, endDate, drive_subfolder):
    """
    Processa um ee.FeatureCollection contendo um ou mais polígonos.
    Cada feição é tratada como um talhão independente, realizando:
      - busca de imagens Sentinel-2
      - associação com probabilidade de nuvem (S2Cloudless)
      - filtragem de imagens sem nuvem sobre o polígono
      - exportação das imagens limpas para o Google Drive

    Parâmetros
    ----------
    name : str
        Nome base do arquivo de entrada (usado para identificar os talhões).
    fc : ee.FeatureCollection
        Coleção de feições (talhões).
    startDate : str
        Data inicial da busca de imagens Sentinel-2.
    endDate : str
        Data final da busca de imagens Sentinel-2.
    drive_subfolder : str
        Subpasta no Google Drive onde os exports serão salvos.
    """

    # Conversão controlada para client-side apenas para iterar feições
    features = fc.getInfo().get('features', [])
    if not features:
        print("Nenhuma feição no FeatureCollection:", name)
        return

    for idx, feat in enumerate(features):
        geom = ee.Geometry(feat['geometry'])
        talhao_name = f"{name}_f{idx}"

        print("Processando talhão:", talhao_name)

        s2 = (
            ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(geom)
            .filterDate(startDate, endDate)
            .sort('system:time_start')
        )

        cloudProbCol = (
            ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(geom)
            .filterDate(startDate, endDate)
        )

        withCloud = add_cloud_probability_join(s2, cloudProbCol)

        # ---------------------------------------------------------------------
        # Avaliação de presença de nuvem sobre o polígono
        # Estratégia: reduceRegion(max) sobre máscara de nuvem
        # ---------------------------------------------------------------------
        def annotate_fn(img):
            cloudMask = img.select('cloud_prob').gt(cloudProbThreshold)

            # Conta pixels com nuvem
            cloud_pixels = cloudMask.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geom,
                scale=10,
                maxPixels=maxPixels
            ).values().get(0)

            # Conta pixels válidos no polígono
            total_pixels = cloudMask.reduceRegion(
                reducer=ee.Reducer.count(),
                geometry=geom,
                scale=10,
                maxPixels=maxPixels
            ).values().get(0)

            # Percentual de nuvem
            cloud_fraction = ee.Number(cloud_pixels).divide(ee.Number(total_pixels))

            # Limite de tolerância
            cloud_free = cloud_fraction.lte(maxAllowedCloudPercentage)

            return img.set({
                'cloud_fraction': cloud_fraction,
                'cloud_free': cloud_free
            })

        annotated = withCloud.map(annotate_fn)

        # Mantém apenas imagens completamente livres de nuvem
        clean = annotated.filter(ee.Filter.eq('cloud_free', 1))



        # try:
        #     total_count = s2.size().getInfo()
        #     clean_count = clean.size().getInfo()
        # except Exception:
        #     total_count, clean_count = 'N/A', 'N/A'

        # print(f"Talhão {talhao_name} -> total scenes: {total_count}, clean scenes: {clean_count}")

        try:
            ids = clean.aggregate_array('system:index').getInfo()
        except Exception:
            ids = []

        if not ids:
            print("Nenhuma imagem limpa para:", talhao_name)
            continue

        print(f"Criando exports para {talhao_name} -> {len(ids)} imagens")


        for img_idx in ids:
            img = ee.Image(clean.filter(ee.Filter.eq('system:index', img_idx)).first())


            img_id = img.get('system:index').getInfo()
            export_name = f"sentinel_{talhao_name}_{img_id}"


            try:
                bounds_info = geom.bounds().getInfo()
                region = bounds_info['coordinates']
            except Exception:
                region = geom.centroid().getInfo().get('coordinates')

            task = ee.batch.Export.image.toDrive(
                image=img.select(bands).clip(geom), # recorte preciso
                description=export_name,
                folder=drive_subfolder,
                fileNamePrefix=export_name,
                region=region,
                scale=10,
                maxPixels=int(maxPixels)
            )
            task.start()
            print("Export task iniciada:", export_name)

    print("Processamento do arquivo", name, "concluído.")


In [ ]:
if not polygons_list:
    print("Nenhum arquivo de polígonos encontrado em", talhoes_dir)
else:
    for name, fc in polygons_list:
        try:

            # cria pasta específica do polígono
            polygon_output_path = os.path.join(base_output_path, name)
            os.makedirs(polygon_output_path, exist_ok=True)

            process_polygon_fc(name, fc, start_Date, end_Date, drive_subfolder=name)

        except Exception as e:
            print("Erro ao processar", name, "->", e)

print("Todas as tasks foram criadas (verifique o painel Tasks no Earth Engine).")

#https://code.earthengine.google.com/tasks
